# Almaty Address NER Project


In [1]:
import spacy
from spacy.training.example import Example
from spacy.scorer import Scorer
from pathlib import Path
import json
import random
import warnings
warnings.filterwarnings("ignore")

## Load and Prepare Labeled Data


In [2]:
data = "street_data.json" 

with open(data, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

def prepare_data(raw_data):
    training_data = []
    for item in raw_data:
        text = item["data"]["Message"]
        entities = []
        if "annotations" in item and item["annotations"]:
            for result in item["annotations"][0]["result"]:
                if result["type"] == "labels":
                    start = result["value"]["start"]
                    end = result["value"]["end"]
                    label = result["value"]["labels"][0]
                    entities.append((start, end, label))
        training_data.append((text, {"entities": entities}))
    return training_data

TRAIN_DATA = prepare_data(raw_data)
print(f"example:\n{TRAIN_DATA[2]}")


example:
('Проспект Назарбаева, дом 12: сломаны лавочки во дворе, жители просят их заменить.', {'entities': [(9, 19, 'Street'), (21, 27, 'NUM')]})


## Load spaCy Model

In [3]:
nlp = spacy.load("ru_core_news_lg")

ner = nlp.get_pipe("ner")

labels = set(ent[2] for _, ann in TRAIN_DATA for ent in ann["entities"])
for label in labels:
    ner.add_label(label)

## Train the NER Model

In [4]:
n_iter = 20
random.seed(42)

optimizer = nlp.resume_training()
for itn in range(n_iter):
    random.shuffle(TRAIN_DATA)
    losses = {}
    for text, annotations in TRAIN_DATA:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.2, losses=losses)
    
    filtered_losses = {k: v for k, v in losses.items() if v > 0.0}
    print(f"epoch {itn + 1}: losses: {filtered_losses}")

epoch 1: losses: {'ner': 1340.3097267156768}
epoch 2: losses: {'ner': 583.9912114208449}
epoch 3: losses: {'ner': 487.0608499647712}
epoch 4: losses: {'ner': 418.66397928905695}
epoch 5: losses: {'ner': 363.0753002317284}
epoch 6: losses: {'ner': 321.1443300927704}
epoch 7: losses: {'ner': 343.3867963652365}
epoch 8: losses: {'ner': 312.82076457388763}
epoch 9: losses: {'ner': 285.1516973822855}
epoch 10: losses: {'ner': 293.8045575847938}
epoch 11: losses: {'ner': 274.6216731064031}
epoch 12: losses: {'ner': 222.99310793635652}
epoch 13: losses: {'ner': 256.7561048326961}
epoch 14: losses: {'ner': 231.41241706260897}
epoch 15: losses: {'ner': 230.44547644777458}
epoch 16: losses: {'ner': 220.8024800546172}
epoch 17: losses: {'ner': 199.83648870134746}
epoch 18: losses: {'ner': 224.9781010879263}
epoch 19: losses: {'ner': 197.0936584606426}
epoch 20: losses: {'ner': 168.20751566865215}


## Inference: Extract Entities from Sample Messages

In [17]:
from IPython.display import Markdown, display

texts = [
    "По ул. Абая возле дома 28 не убирают мусор.",
    "В парке Ганди не хватает урн.",
    "Доброе утро. Пол лета не поливается озеленененный участок по пр. Аблай хана вдоль домов 37-39.",
    "Здравствуйте, на пересечении Толе би Варламова с самого утра горят фонари.",
    "Ветки деревьев очень сильно отросли - улица Тургут Озала, 26",
    "Добрый вечер. Муратбаева Казыбек би отключения электричества продолжается.",
]

for text in texts:
    doc = nlp(text)
    md_text = f"**Message:** {text}  \n"
    for ent in doc.ents:
        md_text += f"&nbsp;&nbsp;&nbsp;&nbsp;{ent.text} → `{ent.label_}`  \n"
    display(Markdown(md_text))


**Message:** По ул. Абая возле дома 28 не убирают мусор.  
&nbsp;&nbsp;&nbsp;&nbsp;Абая → `Street`  
&nbsp;&nbsp;&nbsp;&nbsp;возле дома 28 → `NUM`  


**Message:** В парке Ганди не хватает урн.  
&nbsp;&nbsp;&nbsp;&nbsp;В парке Ганди → `Park`  


**Message:** Доброе утро. Пол лета не поливается озеленененный участок по пр. Аблай хана вдоль домов 37-39.  
&nbsp;&nbsp;&nbsp;&nbsp;Аблай хана → `Street`  
&nbsp;&nbsp;&nbsp;&nbsp;вдоль домов 37-39 → `NUM`  


**Message:** Здравствуйте, на пересечении Толе би Варламова с самого утра горят фонари.  
&nbsp;&nbsp;&nbsp;&nbsp;на пересечении → `Peresechenie`  
&nbsp;&nbsp;&nbsp;&nbsp;Толе би → `Street`  
&nbsp;&nbsp;&nbsp;&nbsp;Варламова → `Street`  


**Message:** Ветки деревьев очень сильно отросли - улица Тургут Озала, 26  
&nbsp;&nbsp;&nbsp;&nbsp;Тургут Озала → `Street`  
&nbsp;&nbsp;&nbsp;&nbsp;26 → `NUM`  


**Message:** Добрый вечер. Муратбаева Казыбек би отключения электричества продолжается.  
&nbsp;&nbsp;&nbsp;&nbsp;Муратбаева → `Street`  
&nbsp;&nbsp;&nbsp;&nbsp;Казыбек би → `Street`  
